In [1]:
#Spark UI
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
# sc master - running locally
sc.master

'local[*]'

# First attempt with distributed systems, using PySpark

In [3]:
# necessary import 
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from pyspark.sql import functions 
from functools import reduce
# create a spark session
spark = SparkSession.builder.appName('SpeciesRecognise').getOrCreate()

In [4]:
import os
from tqdm import tqdm
import PIL.Image
import numpy as np #For numerical functions, incl. statistical.
import matplotlib.pyplot as plt #2D plotting libraryS
import cv2
import pandas as pd
#from keras.utils import np_utils
from keras import utils
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
import tensorflow as tf

2023-09-21 13:38:58.782443: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-21 13:38:58.823121: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-21 13:38:58.823552: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 13:38:59.545007: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Define the HDFS directory

In [5]:
mayfly_directory = '/user1/Species_images/Mayflies_Ephemeroptera/'
stonefly_directory = '/user1/Species_images/Stoneflies_Plecoptera'
caddis_directory = '/user1/Species_images/Trichoptera_caddis_fly'
waterlouse_directory = '/user1/Species_images/Asellus aquaticus  (waterlouse)'

#### Use spark.read to import the images

In [23]:
# load images
mayflies = spark.read.format("image").load(mayfly_directory)                                                                    
stoneflies = spark.read.format("image").load(stonefly_directory)
caddis_flies = spark.read.format("image").load(caddis_directory)
waterlouse = spark.read.format("image").load(waterlouse_directory)

In [7]:
images = [mayflies, stoneflies, caddis_flies, waterlouse]
#labels = list(np.repeat(["mayflies", "stoneflies", "caddis_flies", "waterlouse"],10))

# merge data frame
df = reduce(lambda first, second: first.union(second), images)
# repartition dataframe 
X = df.repartition(200)

#### The below shows that a Spark schema is created with the images as data

In [8]:
X.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



#### The count matches the number of images - 10 of each of the four inter-class groups

In [9]:
print((X.count(), len(X.columns)))

(40, 1)


#### Showing the Spark dataframe type

In [10]:
print(type(X))

<class 'pyspark.sql.dataframe.DataFrame'>


#### Can't select any of the parameters of the images because it looks like they have been renamed, and therefore cannot now be found.

In [11]:
X.select("image.origin", "image.width", "image.height").show(truncate=False)

2023-09-21 13:39:05,411 ERROR executor.Executor: Exception in task 10.0 in stage 6.0 (TID 251)
java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Stoneflies_Plecoptera/stonefly%202.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslatorPB.java:442)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)
	at 

2023-09-21 13:39:05,677 WARN scheduler.TaskSetManager: Lost task 6.0 in stage 6.0 (TID 247) (192.168.1.117 executor driver): TaskKilled (Stage cancelled)
2023-09-21 13:39:05,678 WARN scheduler.TaskSetManager: Lost task 8.0 in stage 6.0 (TID 249) (192.168.1.117 executor driver): TaskKilled (Stage cancelled)
2023-09-21 13:39:05,678 WARN scheduler.TaskSetManager: Lost task 0.0 in stage 6.0 (TID 241) (192.168.1.117 executor driver): TaskKilled (Stage cancelled)
2023-09-21 13:39:05,679 WARN scheduler.TaskSetManager: Lost task 9.0 in stage 6.0 (TID 250) (192.168.1.117 executor driver): TaskKilled (Stage cancelled)
2023-09-21 13:39:05,679 WARN scheduler.TaskSetManager: Lost task 5.0 in stage 6.0 (TID 246) (192.168.1.117 executor driver): TaskKilled (Stage cancelled)
2023-09-21 13:39:05,680 WARN scheduler.TaskSetManager: Lost task 7.0 in stage 6.0 (TID 248) (192.168.1.117 executor driver): TaskKilled (Stage cancelled)
2023-09-21 13:39:05,680 WARN scheduler.TaskSetManager: Lost task 2.0 in stag

Py4JJavaError: An error occurred while calling o61.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 6.0 failed 1 times, most recent failure: Lost task 10.0 in stage 6.0 (TID 251) (192.168.1.117 executor driver): java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Stoneflies_Plecoptera/stonefly%202.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslatorP

#### Attempted to cache the dataframe to prevent the above issue

In [12]:
X.cache()

DataFrame[image: struct<origin:string,height:int,width:int,nChannels:int,mode:int,data:binary>]

In [13]:
X.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [14]:
X.show()

2023-09-21 13:44:44,638 ERROR executor.Executor: Exception in task 3.0 in stage 7.0 (TID 259)
java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Mayflies_Ephemeroptera/Ecdyonurus%20dispar%2010.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslatorPB.java:442)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProto

Py4JJavaError: An error occurred while calling o53.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 7.0 failed 1 times, most recent failure: Lost task 3.0 in stage 7.0 (TID 259) (192.168.1.117 executor driver): java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Mayflies_Ephemeroptera/Ecdyonurus%20dispar%2010.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSide

In [15]:
print((X.count(), len(X.columns)))

2023-09-21 13:44:51,028 ERROR executor.Executor: Exception in task 3.0 in stage 9.0 (TID 272)
java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Mayflies_Ephemeroptera/Ecdyonurus%20dispar%2010.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslatorPB.java:442)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProto

Py4JJavaError: An error occurred while calling o53.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 9.0 failed 1 times, most recent failure: Lost task 3.0 in stage 9.0 (TID 272) (192.168.1.117 executor driver): java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Mayflies_Ephemeroptera/Ecdyonurus%20dispar%2010.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTrans

In [16]:
X.select("image.origin", "image.width", "image.height").show(truncate=False)

2023-09-21 13:44:55,535 ERROR executor.Executor: Exception in task 10.0 in stage 11.0 (TID 292)
java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Stoneflies_Plecoptera/stonefly%202.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslatorPB.java:442)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)
	at

Py4JJavaError: An error occurred while calling o79.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 11.0 failed 1 times, most recent failure: Lost task 10.0 in stage 11.0 (TID 292) (192.168.1.117 executor driver): java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Stoneflies_Plecoptera/stonefly%202.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslato

In [17]:
images_list = X.select('image').collect()

2023-09-21 13:44:59,464 ERROR executor.Executor: Exception in task 10.0 in stage 13.0 (TID 305)
java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Stoneflies_Plecoptera/stonefly%202.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslatorPB.java:442)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)
	at

Py4JJavaError: An error occurred while calling o85.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 13.0 failed 1 times, most recent failure: Lost task 10.0 in stage 13.0 (TID 305) (192.168.1.117 executor driver): java.io.FileNotFoundException: 
File does not exist: /user1/Species_images/Stoneflies_Plecoptera/stonefly%202.jpeg
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2002)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:773)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTran

## Second attempt at PySpark

In [1]:
#Packages failing to import, not going to spend more time on it

from pyspark.ml.image import ImageSchema
#from sparkdl.image import imageIO
#from sparkdl import DeepImagePredictor

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
mayfly_df = ImageSchema.readImages(mayfly_directoryS

predictor = DeepImagePredictor(inputCol="image", outputCol="predicted_labels", modelName="VGG16", decodePredictions=True, topK=5)
predictions_df = predictor.transform(mayfly_df).cache()
predictions_df.count()

# Second attempt to use distributed systems with PyTorch

In [1]:
import os
from PIL import Image
import torch
from torchvision import transforms
import torchvision.models as models
from tqdm import tqdm
import numpy as np

In [2]:
mayfly_directory = os.getcwd() + '/Species_images/Mayflies_Ephemeroptera/'
mayfly_resized = os.getcwd() + '/Species_images/Ephemeroptera_resized/'
stonefly_directory = os.getcwd() + '/Species_images/Stoneflies_Plecoptera'
stonefly_resized = os.getcwd() + '/Species_images/Stoneflies_resized/'
caddis_directory = os.getcwd() + '/Species_images/Trichoptera_caddis_fly'
caddis_resized = os.getcwd() + '/Species_images/Trichoptera_resized'
Waterlouse_directory = os.getcwd() + '/Species_images/Asellus aquaticus  (waterlouse)'
Waterlouse_resized = os.getcwd() + '/Species_images/Asellus_resized'

In [3]:
# needed input dimensions for the CNN
input_dim = (250,250)

transformation_for_cnn_input = transforms.Compose([transforms.Resize(input_dim)])

for image_name in os.listdir(mayfly_directory):
    I = Image.open(os.path.join(mayfly_directory, image_name))
    newI = transformation_for_cnn_input(I)
    newI.save(os.path.join(mayfly_resized, image_name))
    newI.close()
    I.close()

for image_name in os.listdir(stonefly_directory):
    I = Image.open(os.path.join(stonefly_directory, image_name))
    newI = transformation_for_cnn_input(I)
    newI.save(os.path.join(stonefly_resized, image_name))
    newI.close()
    I.close()
    
for image_name in os.listdir(caddis_directory):
    I = Image.open(os.path.join(caddis_directory, image_name))
    newI = transformation_for_cnn_input(I)
    newI.save(os.path.join(caddis_resized, image_name))
    newI.close()
    I.close()
    
for image_name in os.listdir(Waterlouse_directory):
    I = Image.open(os.path.join(Waterlouse_directory, image_name))
    newI = transformation_for_cnn_input(I)
    newI.save(os.path.join(Waterlouse_resized, image_name))
    newI.close()
    I.close()

## As per sources, V66-16 is a commonly and successfully used CNN model for species identification tasks

### Testing a pre-trained VGG model (this is not expected to be accurate, as the dataset has not been trained on these Genera) - transfer learning

In [4]:
vgg16 = models.vgg16(pretrained=True)

/home/hduser/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hduser/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
#Function to tranform image to tensor
transform = transforms.Compose([
              transforms.ToTensor()
            ])

In [6]:
# Set the model to evaluation mode
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
#Testing the pretrained VGG with 4 genera
directories = [mayfly_resized, stonefly_resized, caddis_resized, Waterlouse_resized]
genus=[]
prediction = []
i=0

for directory in directories:
    for img in tqdm(os.listdir(directory)):
        genus.append(img)
        img = Image.open(os.path.join(directory, img))
        img = transform(img)
        img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
        with torch.no_grad():
            output = vgg16(img)

        # Get the index of the predicted class
        pred_idx = torch.argmax(output)

        # Get the label for the predicted class
        labels = open("imagenet_classes.txt").read().splitlines()
        label = labels[pred_idx]

        prediction.append(label)
        i+=1


100%|███████████████████████████████████████████| 10/10 [00:02<00:00,  3.65it/s]


### As expected, the predictions are incorrect becuase the pretrained VGG16 has not been trained on these genera

In [8]:
np.column_stack((genus, prediction))

array([['Ecdyonurusdispar9.jpeg', '122, American_lobster'],
       ['Ecdyonurusdispar7.jpeg', '122, American_lobster'],
       ['Ecdyonurus dispar 10.jpeg', '122, American_lobster'],
       ['Ecdyonurusdispar4.jpeg', '724, pirate'],
       ['Ecdyonurusdispar8.jpeg', '122, American_lobster'],
       ['Ecdyonurusdispar2.jpeg', '122, American_lobster'],
       ['Ecdyonurusdispar5.jpeg', '122, American_lobster'],
       ['Ecdyonurusdispar6.jpeg', '122, American_lobster'],
       ['Ecdyonurusdispar.jpeg', '122, American_lobster'],
       ['Ecdyonurusdispar3.jpeg', '122, American_lobster'],
       ['stonefly 10.jpeg', '306, rhinoceros_beetle'],
       ['stonefly 8.jpeg', '122, American_lobster'],
       ['stonefly 1.jpeg', '122, American_lobster'],
       ['stonefly 6.jpeg', '673, mouse'],
       ['stonefly 5.jpeg', '122, American_lobster'],
       ['stonefly 9.jpeg', '122, American_lobster'],
       ['stonefly 4.jpeg', '122, American_lobster'],
       ['stonefly 2.jpeg', '122, American_lobs